In [2]:
import sys
from pathlib import Path
sys.path.insert(0, str(Path.cwd().parent)) 
from tool_monkey import MonkeyObserver, with_monkey, retry_exhaustion, logger, setup_default_logging

In [3]:
setup_default_logging(level=10)

### Example 1: Retry Exhaustion with Tenacity

**Use Case:** Weather API consistently times out due to server overload. Retry logic (using tenacity) exhausts all attempts.

**What Happens:**
1. User asks "What's the weather in Boston?"
2. LLM calls `get_weather("Boston")`
3. **Attempt 1:** Times out after 2 seconds
4. **Tenacity retry (attempt 2):** Times out again after 2 seconds
5. **Tenacity retry (attempt 3):** Times out again after 2 seconds
6. **All retries exhausted** - fallback error message returned
7. Agent tells user: "Weather data unavailable"

**What This Shows:**
- Even with retry logic, some failures are persistent (server down, quota exhausted, etc.)
- Tenacity's `retry_error_callback` provides graceful fallback message
- Proper error handling pattern: retry N times, then give up gracefully
- Real pattern: Don't retry forever - set limits

**Retry Strategy:**
- `stop_after_attempt(3)` - max 3 attempts
- `wait_exponential_jitter(initial=0.2, max=2.0)` - exponential backoff with jitter
- `retry_error_callback` - returns user-friendly error message when exhausted

**Expected Output:**
- 3 timeout failures in a row
- Observer shows: 3 calls, 0% success rate, 3 failures, 3 retries
- Agent receives error message and informs user gracefully

In [ ]:
def timeout_retry_example():
    from langchain_examples.shared.llm import llm
    from langchain_examples.shared.tools import base_weather_tool 
    from langchain_core.tools import tool
    from tenacity import retry, stop_after_attempt, wait_exponential_jitter
    observer=MonkeyObserver()
    scenario=retry_exhaustion(num_failures=3, seconds=2)

    # first wrap the base tool with the monkey decorator
    wrapped_weather_tool=with_monkey(scenario, observer)(base_weather_tool)
    # then wrap it with tenacity retry logic
    @retry(
        stop=stop_after_attempt(3),
        wait=wait_exponential_jitter(initial=0.2, max=2.0),
        retry_error_callback=lambda retry_state: "Weather data unavailable due to timeout",
    )
    def weather_tool_with_retry(location:str, units:str="celcius"):
        return wrapped_weather_tool(location, units)
    
    # finally define the tool implementation
    @tool
    def get_weather(location: str, units: str = "celcius"):
        """Get the current weather for a given location.
            Args:
                location (str): The location to get the weather for.
                units (str): The units to return the weather in. Either 'celsius' or 'fahrenheit'.
            Returns:
                str: The current weather in the given location."""
        return weather_tool_with_retry(location, units)
    
    messages = [
    {"role": "system", "content": "You are a helpful assistant with access to a tool to get weather information."},
    {"role": "user", "content": "What's the weather in Boston?"}]
    llm_with_tool = llm.bind_tools([get_weather])
    # invoke the model with the messages
    ai_msg = llm_with_tool.invoke(messages)
    logger.debug(f"Full LLM Response: {ai_msg}")
    messages.append(ai_msg)
    logger.debug(f"Have tool calls? {'Yes' if ai_msg.tool_calls else 'No'}")
    try:
        for tool_call in ai_msg.tool_calls:
            logger.debug(
                f"Invoking tool: {tool_call.get("name")} with args {tool_call.get('args')}")
            tool_result = get_weather.invoke(tool_call)
            logger.debug(f"Tool result: {tool_result}")
            messages.append(tool_result)
        final_response = llm_with_tool.invoke(messages)
        logger.debug(f"Final LLM Response: {final_response.text}")
    except Exception:
        pass
    print("\n" + "=" * 50)
    print("OBSERVER METRICS:")
    print("=" * 50)
    print(observer.summary())

timeout_retry_example()

DEBUG: Full LLM Response: content='' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 15, 'prompt_tokens': 134, 'total_tokens': 149, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-4-0613', 'system_fingerprint': None, 'id': 'chatcmpl-CwvgVoDydLStru63AtrYmXnW5TK5e', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None} id='lc_run--019bae98-a99f-79c2-95d0-bbec972320b6-0' tool_calls=[{'name': 'get_weather', 'args': {'location': 'Boston'}, 'id': 'call_sHKxecQ1tbKetxWGuJqeZmOk', 'type': 'tool_call'}] invalid_tool_calls=[] usage_metadata={'input_tokens': 134, 'output_tokens': 15, 'total_tokens': 149, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}}
DEBUG: Have tool cal

Ending call for base_weather_tool after exception
Ending call for base_weather_tool


INFO: Simulating timeout for 2.0 seconds


Ending call for base_weather_tool after exception
Ending call for base_weather_tool


INFO: Simulating timeout for 2.0 seconds
DEBUG: Tool result: content='Weather data unavailable due to timeout' name='get_weather' tool_call_id='call_sHKxecQ1tbKetxWGuJqeZmOk'


Ending call for base_weather_tool after exception
Ending call for base_weather_tool


DEBUG: Final LLM Response: I'm sorry, but I'm currently unable to retrieve the weather information for Boston due to a technical issue. Please try again later.



OBSERVER METRICS:
Tool Monkey Execution Summary
  Total Calls: 3
  Success Rate: 0.0%
  Failures: 3
  Total Retries: 3
  Avg Latency: 2001.7ms


### Example 2: ReAct Agent with retry exhaustion

In [ ]:
def ecommerce_react_example():
    from langchain_examples.shared.llm import llm
    from langchain_examples.shared.tools import base_place_order, base_check_inventory, base_search_products # mock functions 
    from langchain_core.tools import tool
    observer=MonkeyObserver()
    scenario=retry_exhaustion(num_failures=3, seconds=3)
    wrapped_tool=with_monkey(scenario, observer)(base_check_inventory)
    def define_tools():
        @tool
        def check_inventory(product_id:str, quantity:int):
            """Check the inventory for a product id and a given quantity. 
            Assumes quantity is 1 if unspecified by user
            Args:
                product_id (str): The product id to check inventory for.
                quantity (int): The quantity to check inventory for.
            """
            return wrapped_tool(product_id, quantity)
        @tool
        def search_products(query:str, category: str):
            """Search for products matching the given query in a category.
            Args:
                query (str): The search query.
                category (str): The product category to search in.
            """
            return base_search_products(query, category) # doesnt fail
        @tool
        def place_order(product_id:str, quantity:int, customer_email: str):
            """Place an order for a product id, quantity, and send email to customer.
            Args:
                product_id (str): The product id to place an order for.
                quantity (int): The quantity to order.
                customer_email (str): The email address of the customer.
            """
            return base_place_order(product_id, quantity, customer_email)
        tools=[check_inventory, search_products, place_order]
        return tools
    tools=define_tools()
    system_msg="""
  You are a helpful e-commerce assistant. When a user wants to buy something:
  1. First, search for products using search_products. The available product categories are electronics, books, and clothing.
  2. Then, check inventory using check_inventory to see if it's in stock
  3. Finally, place the order using place_order

  Always check inventory before placing an order.
"""
    user_msg="I want to buy a laptop. My email is customer@example.com"
    messages=[
        {"role":"system", "content":system_msg},
        {"role":"user", "content":user_msg}
    ]
    llm_with_tools=llm.bind_tools(tools)
    max_iterations=10
    for i in range(max_iterations):
        ai_msg=llm_with_tools.invoke(messages)
        messages.append(ai_msg)
        if not ai_msg.tool_calls:
            print(f"ReAct Loop finished: {ai_msg.content}")
            break
        for tool_call in ai_msg.tool_calls:
            tool_name=tool_call["name"]
            tool_args=tool_call["args"]
            logger.debug(
                f"Invoking tool: {tool_name} with args {tool_args}")
            try:
                if tool_name == "search_products":
                    func=next(t for t in tools if t.name == "search_products")
                    result = func.invoke(tool_call)
                    messages.append(result)
                    print(f"✅ Searched products")
                elif tool_name == "check_inventory":
                    func=next(t for t in tools if t.name == "check_inventory")
                    result = func.invoke(tool_call)
                    messages.append(result)
                    print(f"✅ Checked inventory")
                elif tool_name == "place_order":
                    func=next(t for t in tools if t.name == "place_order")
                    result = func.invoke(tool_call)
                    messages.append(result)
                    print(f"✅ Placed order")
            except TimeoutError as e:
                print(f"❌ Timeout: {e}")
                messages.append({
                    "role": "tool",
                    "tool_call_id": tool_call["id"],
                    "content": f"Error: {e}"
                })
    print("\n" + "=" * 50)
    print("OBSERVER METRICS:")
    print("=" * 50)
    print(observer.summary())
ecommerce_react_example()